<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/Tokenizer_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torchtext==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 431 kB 64.4 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
     |████████████████████████████████| 163 kB 70.5 MB/s 
     |████████████████████████████████| 115 kB 52.8 MB/s 
     |████████████████████████████████| 212 kB 56.1 MB/s 
     |████████████████████████████████| 127 kB 50.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
#from torchtext.legacy.datasets import Multi30k
#from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import os
import re
import json
import pickle
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

import pandas as pd

data = pd.read_csv("drive/MyDrive/AIHUB_일상생활및구어체_DATAPAIR/AIHUB_일상생활_val_15만PAIR.csv", encoding='utf-8-sig')
data.head(100)


data = data[:150000]

In [ ]:
from transformers import CLIPTokenizer
from transformers import AutoTokenizer

#CLIPTokenizer == Tokenizer that StableDiffusion used.
en_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
ko_tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

def tokenize_ko(text):
    """
    Tokenizes KOREAN text from a string into a list of strings
    """
    return ko_tokenizer.encode(text, padding="max_length", max_length=48, truncation=True, return_tensors="pt")
    #return ko_tokenizer.encode(text, padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0].tolist()

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return en_tokenizer.encode(text, padding="max_length", max_length=48, truncation=True, return_tensors="pt")
    #return en_tokenizer.encode(text, padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0].tolist()
   #return [tok.text for tok in spacy_en.tokenizer(text)]


print(tokenize_ko("난 너를 사랑해 -호반우-"))


print(tokenize_en("I RESPECT YOU i respect you cow, -hobanu-"))

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


tensor([[    0,   720,   743,  2138,  3784,  2097,    17, 19803,  2137,    17,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1]])
tensor([[49406,   328,  4916,   592,   328,  4916,   592,  9706,   267,   268,
           606,  1159,   340,   268, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407]])


In [ ]:
data.head()

,en,ko
0,"Features include heart rate monitoring, 14-day...","심박수 모니터링, 14일 배터리 수명, 음악 플레이 제어 디스플레이 수면 및 수영 ..."
1,The touch screen has 3rd generation tempered g...,터치스크린에는 3세대 강화유리와 지문 방지 코팅이 적용되었습니다.
2,"For product inquiries, please send us an email.",제품 문의는 이메일로 주시면 감사하겠습니다.
3,"Hello, BBB company AAA.","안녕하세요, BBB회사 AAA입니다."
4,I would like to introduce the Ultimate Radianc...,저는 얼티밋 래디언스 컬렉션 스킨케어 선물 세트 제품을 소개하려고 합니다.


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   en      150000 non-null  object
 1   ko      150000 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [ ]:

data.head(100)

,en,ko
0,"Features include heart rate monitoring, 14-day...","심박수 모니터링, 14일 배터리 수명, 음악 플레이 제어 디스플레이 수면 및 수영 ..."
1,The touch screen has 3rd generation tempered g...,터치스크린에는 3세대 강화유리와 지문 방지 코팅이 적용되었습니다.
2,"For product inquiries, please send us an email.",제품 문의는 이메일로 주시면 감사하겠습니다.
3,"Hello, BBB company AAA.","안녕하세요, BBB회사 AAA입니다."
4,I would like to introduce the Ultimate Radianc...,저는 얼티밋 래디언스 컬렉션 스킨케어 선물 세트 제품을 소개하려고 합니다.
...,...,...
95,"And it features a textured rose gold bangle, a...","그리고 이 제품은 투명한 1개의 크리스탈로 장식된, 질감을 살린 로즈골드 뱅글을 갖..."
96,This product is an adjustable end link.,이 제품은 조절식 엔드 링크입니다.
97,And there are jewelry buckles and extension co...,그리고 주얼리 버클과 연장줄이 있습니다.
98,"For product inquiries, please send us an email.",제품 문의는 이메일로 주시면 감사하겠습니다.


In [ ]:
data['en'][0]

'Features include heart rate monitoring, 14-day battery life, music play control display sleep and swim tracking waterproof smart notifications.'

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

from datasets import Dataset
import pandas as pd 


In [ ]:
jc = Dataset.from_pandas(train_df)
jc

Dataset({
    features: ['en', 'ko', '__index_level_0__'],
    num_rows: 120000
})

In [ ]:
#train_dataset = Dataset.from_pandas(train_df).remove_columns(['__index_level_0__'])
#val_dataset = Dataset.from_pandas(val_df).remove_columns(['__index_level_0__'])
import torch
dev=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#train_dataset = Dataset.from_pandas(train_df).remove_columns(['__index_level_0__']).with_format("torch", device=dev)
#val_dataset = Dataset.from_pandas(val_df).remove_columns(['__index_level_0__']).with_format("torch", device=dev)
train_dataset = Dataset.from_pandas(train_df).remove_columns(['__index_level_0__'])
val_dataset = Dataset.from_pandas(val_df).remove_columns(['__index_level_0__'])
val_dataset



Dataset({
    features: ['en', 'ko'],
    num_rows: 30000
})

In [ ]:
import torch
from torch.utils.data import dataset

#코딩 에이징커브가 와서 src가 ko였는지 en이었는지 계속 헷갈리는 중... In my code -> SRC: KO
def data_tokenize(text_iter: dataset.IterableDataset) -> torch.Tensor:
    """Src(source) and Tgt(target), convert raw texts into a Tensor.""" 
    src_data = [ko_tokenizer.encode(item['ko'], padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0] for item in text_iter]
    tgt_data = [en_tokenizer.encode(item['en'], padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0] for item in text_iter]
    
#    return torch.cat(src_data), torch.cat(tgt_data)
    return src_data, tgt_data

ko_tokens, en_tokens  = data_tokenize(train_dataset)
ko_tokens_val, en_tokens_val  = data_tokenize(val_dataset)

en_tokens_val


[tensor([49406,   328,  1311,   789,   531,  9813,   320,  1929,  5988,  4306,
           269, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407]),
 tensor([49406,  1071, 44990,   612,   753,   655,   515,  2026,   537, 46791,
           827, 35321, 11639,  1737,   267, 22134,   267,   537,  9273, 24311,
           269, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407]),
 tensor([49406,   328,   720,  1446, 40470,   592,   518,  1476, 39705,   539,
           581, 11108,   269, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 4940

In [ ]:
print(len(en_tokens_val))
print(len(en_tokens))

30000
120000


In [ ]:
koko = torch.stack(ko_tokens)

koko.shape

torch.Size([120000, 48])

In [ ]:
koko[0]

tensor([    0,   831,   717,  2073,  3838,  2138,  3894,  2205, 19644,  3916,
         2052,  4625,  2205,  2259,   575,  2052, 16680,  8969,  3794,  2085,
         1295,  1513, 13091,  6159,  6422,  2259,  3731,  2138,  3894,  2085,
          575, 12190,    18,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1])

In [ ]:
ko_tokens[3333]

tensor([   0, 4867, 2170, 3618, 7775, 2052, 3915, 2496, 2359, 2219, 3606,   18,
           2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

#train_dataset = list(zip(ko_tokens, en_tokens))
#val_dataset = list(zip(ko_tokens_val, en_tokens_val))
train_dataset = TensorDataset(torch.stack(ko_tokens).cuda(), torch.stack(en_tokens).cuda())
val_dataset = TensorDataset(torch.stack(ko_tokens_val).cuda(), torch.stack(en_tokens_val).cuda())

train_iter = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_iter = DataLoader(val_dataset, batch_size=128, shuffle=True)
val_iter

In [ ]:
print(len(list(val_iter)))
print(len(list(val_iter)[0]))
print(len(list(val_iter)[0][0]))
print(len(list(val_iter)[0][0][0]))
#print(len(list(val_iter)[0][0][0][0]))

235
2
128
48


In [ ]:
list(val_iter)[234][0][2]

tensor([    0,    23,  2174,  1588,  2295,  2079,    24,  2213, 18283,  5677,
         2259,  4262,  2897, 10489,  2138,  8871, 11800,    18,     2,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1],
       device='cuda:0')

In [ ]:
list(val_iter)[23][1][99]

tensor([49406,   589,  4306,   533,   556,  1753,  6933,  3575,   533,  2530,
          531,  5354, 27479,  7243,   537, 16936,  1294,   269, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407],
       device='cuda:0')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128


## Vanilla Transformer

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention



##MODEL FIT

In [ ]:
james=[[1,2,3,4],[1,2,3,4],[1,2,3,4]]

james[:]

[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]

In [ ]:
#INPUT_DIM = len(SRC.vocab)
#OUTPUT_DIM = len(TRG.vocab)
INPUT_DIM = 32000 #KO TOK
OUTPUT_DIM = 49408 #EN TOK
HID_DIM = 256
ENC_LAYERS = 3 # MODEL_ZOO_SIZE = SMALL
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

#weight initialization (Xavier)
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

#SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
SRC_PAD_IDX = 1
TRG_PAD_IDX = 49407

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

model.apply(initialize_weights)
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0]
        trg = batch[1]
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1]

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bestsaved-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 2m 45s
	Train Loss: 3.054 | Train PPL:  21.195
	 Val. Loss: 2.694 |  Val. PPL:  14.795
Epoch: 02 | Time: 2m 45s
	Train Loss: 2.382 | Train PPL:  10.826
	 Val. Loss: 2.313 |  Val. PPL:  10.104
Epoch: 03 | Time: 2m 45s
	Train Loss: 1.964 | Train PPL:   7.127
	 Val. Loss: 2.074 |  Val. PPL:   7.960
Epoch: 04 | Time: 2m 45s
	Train Loss: 1.679 | Train PPL:   5.362
	 Val. Loss: 1.945 |  Val. PPL:   6.997
Epoch: 05 | Time: 2m 45s
	Train Loss: 1.475 | Train PPL:   4.370
	 Val. Loss: 1.861 |  Val. PPL:   6.432
Epoch: 06 | Time: 2m 45s
	Train Loss: 1.319 | Train PPL:   3.741
	 Val. Loss: 1.809 |  Val. PPL:   6.106
Epoch: 07 | Time: 2m 45s
	Train Loss: 1.199 | Train PPL:   3.315
	 Val. Loss: 1.776 |  Val. PPL:   5.907
Epoch: 08 | Time: 2m 45s
	Train Loss: 1.104 | Train PPL:   3.016
	 Val. Loss: 1.777 |  Val. PPL:   5.911
Epoch: 09 | Time: 2m 45s
	Train Loss: 1.026 | Train PPL:   2.790
	 Val. Loss: 1.767 |  Val. PPL:   5.854
Epoch: 10 | Time: 2m 45s
	Train Loss: 0.959 | Train PPL

In [ ]:
model.load_state_dict(torch.load('bestsaved-model.pt'))

#test_loss = evaluate(model, test_iter, criterion)
test_loss = evaluate(model, val_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 1.768 | Test PPL:   5.858 |


## inference

In [ ]:
def translate_sentence(sentence, model, device, max_len = 48):
    
    model.eval()

    '''    
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    '''
    #tokens = [ko_tokenizer.encode(item['ko'], padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0] for item in text_iter]
    src_indexes = ko_tokenizer.encode(sentence, padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0]
    #tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    #src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [49406] #init token

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        #if pred_token == 49407: #end token
        #    break
    
    trg_tokens = trg_indexes
    
    return trg_tokens, attention

def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [ ]:
example_idx = 8

src = train_df['ko'][8]
trg = train_df['en'][8]

print(f'src = {src}')
print(f'trg = {trg}')

src = 수분이 풍부한 피부 깊은 곳에서 스며 나오는 광채를 선사합니다.
trg = Provides a radiance that seeps from the depths of the hydrated skin.


In [ ]:
translation, attention = translate_sentence(src, model, device)

print(f'predicted trg = {translation}')

NameError: ignored

In [ ]:
src_indexes = ko_tokenizer.encode(src, padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0]

src_indexes    

tensor([    0,  7349,  2052,  6007,  2470,  4335,   652,  2073,   601, 27135,
        11854,  3853,  2259, 21125,  2138,  6662, 11800,    18,     2,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1])

In [ ]:
tgt_indexes = en_tokenizer.encode(trg, padding="max_length", max_length=48, truncation=True, return_tensors="pt")[0]
tgt_indexes

tensor([49406,  7161,   320, 45670,   682,  1751,   814,   633,   518, 28855,
          539,   518, 23300,  3575,   269, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407])

In [ ]:
en_tokenizer.decode(translation)

'<|startoftext|>it provides a radiance that seeps from the depths of the moisture. m.. coli. to sleep............ it leaves it off the radiance...... it out the'